# Annotating the DHS Masterlist with Gencode

1. Download Gencode Annotation file and DHS Masterlist file
2. Parse Gencode file and create bed files for gene body, exon, cds , promoter, utr, intronic, and intergenic
3. Map promoter, exon, intronic, and intergenic regions to DHS Masterlist
4. Annotate the DHS as promoter, exon, intron, or intergenic depending on which element had the largest overlap. If there is a tie in overlap, the winning annotation is based on the rank
5. For CDS and UTR under exon, pick the element that has the largest overlap. If there was a tie, pick the largest fraction of overlap
6. For Protein-Coding and Non-Protein-Coding under promoter/intronic, pick the element that has the largest overlap. If there was a tie, pick the largest fraction of overlap


## Download Gencode, DHS Masterlist, and genome files

1. Look at Github README.md for instructions on how to download and name Gencode Annotation file and DHS Masterlist
2. Download chromInfo.hg38.bed from Github repository
3. Load bedops in order to run bedops, bedmap, and sort-bed

### DHS Masterlist

In [55]:
%%bash
#Initial DHS_Index Filename should be "DHS_Index_and_Vocabulary_hg38_WM20190703.txt.gz"
gunzip DHS_Index_and_Vocabulary_hg38_WM20190703.txt.gz
cut -f1-3 DHS_Index_and_Vocabulary_hg38_WM20190703.txt \
| tail -n +2 \
> DHS_Index.bed

head DHS_Index.bed
total=`wc -l DHS_Index.bed | cut -d' ' -f1`
echo "Total number of rows: $total"

chr1	16140	16200
chr1	51868	52040
chr1	57280	57354
chr1	66370	66482
chr1	79100	79231
chr1	79430	79497
chr1	79580	79760
chr1	87220	87295
chr1	88220	88360
chr1	88700	88814
Total number of rows: 3591898


### Gencode

In [57]:
%%bash
#Initial Gencode Filename should be "gencode.v28.basic.annotation.gtf.gz"
gunzip gencode.v28.basic.annotation.gtf.gz
head gencode.v28.basic.annotation.gtf

##description: evidence-based annotation of the human genome (GRCh38), version 28 (Ensembl 92)
##provider: GENCODE
##contact: gencode-help@ebi.ac.uk
##format: gtf
##date: 2018-03-23
chr1	HAVANA	gene	11869	14409	.	+	.	gene_id "ENSG00000223972.5"; gene_type "transcribed_unprocessed_pseudogene"; gene_name "DDX11L1"; level 2; havana_gene "OTTHUMG00000000961.2";
chr1	HAVANA	transcript	11869	14409	.	+	.	gene_id "ENSG00000223972.5"; transcript_id "ENST00000456328.2"; gene_type "transcribed_unprocessed_pseudogene"; gene_name "DDX11L1"; transcript_type "processed_transcript"; transcript_name "RP11-34P13.1-002"; level 2; transcript_support_level "1"; tag "basic"; havana_gene "OTTHUMG00000000961.2"; havana_transcript "OTTHUMT00000362751.1";
chr1	HAVANA	exon	11869	12227	.	+	.	gene_id "ENSG00000223972.5"; transcript_id "ENST00000456328.2"; gene_type "transcribed_unprocessed_pseudogene"; gene_name "DDX11L1"; transcript_type "processed_transcript"; transcript_name "RP11-34P13.1-002"; exon_number 1; e

## Parse Gencode and map to DHS Masterlist

In [58]:
%%bash
#Remove row if start = end (only ~1100 cases)
tail -n +6 gencode.v28.basic.annotation.gtf \
| awk -F'\t' '{
    if($4 != $5) {
        print $1"\t"$4"\t"$5"\t"$3"\t"$7
    }
}' \
| sort-bed -  \
> gencode.v28.basic.annotation.filtered.gtf

#Expand the transcription region to say promoter. +/- 1KB of TSS
awk -F'\t' '{
        if($4 == "transcript") {
                if ($5 == "+") {
                        print $1"\t"$2"\t"$2+1000"\t""promoter";
                }
                else if ($5 == "-") {
                        print $1"\t"$3-1000"\t"$3"\t""promoter";
                }
    }
        else if($4 != "transcript") {
                print $1"\t"$2"\t"$3"\t"$4;
        }
}' gencode.v28.basic.annotation.filtered.gtf \
| grep -v chrM | grep -v Selenocysteine | grep -v codon \
| sort-bed - \
> gencode.bed

#Need to find the INTRONS. Difference between gene and (CDS + PROMOTER + UTR) 
awk '{if($4 == "gene") print}' gencode.bed > gene.bed
awk '{if($4 == "exon") print}' gencode.bed > exon.bed
awk '{if($4 == "CDS") print}' gencode.bed > cds.bed
awk '{if($4 == "promoter") print}' gencode.bed > promoter.bed
awk '{if($4 == "UTR") print}' gencode.bed > utr.bed

bedops --ec -m utr.bed exon.bed promoter.bed cds.bed | bedops --ec -d gene.bed - > tmp.intron.bed
awk '{print $1"\t"$2"\t"$3"\t""intron"}' tmp.intron.bed > intron.bed

#Need to find the Intergenic region. Difference between Genome and gene-body + promoter region
cat chromInfo.hg38.bed  \
| grep -v chrM > chromInfoNoM.bed
bedops --ec -d chromInfoNoM.bed gene.bed promoter.bed > tmp.intergenic.bed
awk '{print $1"\t"$2"\t"$3"\t""intergenic"}' tmp.intergenic.bed > intergenic.bed

#Clean Up
rm tmp.intron.bed
rm tmp.intergenic.bed

#Unite promoter, exon, intron, and intergenic regions in one bed file
#Map united bed file and map to DHS_Index.bed
bedops --ec -u promoter.bed exon.bed intron.bed intergenic.bed | sort-bed - > gencode-genome.bed
bedmap --ec --echo --echo-map --skip-unmapped --echo-overlap-size DHS_Index.bed gencode-genome.bed > gencode_mapped.bed

### Bed file of parsed Gencode
Includes rows that overlap

In [59]:
%%bash
head gencode-genome.bed

chr1	0	11869	intergenic
chr1	11869	12227	exon
chr1	11869	12869	promoter
chr1	12010	12057	exon
chr1	12010	13010	promoter
chr1	12179	12227	exon
chr1	12613	12697	exon
chr1	12613	12721	exon
chr1	12975	13052	exon
chr1	13052	13221	intron


### Bed file of Gencode mapped to DHS Masterlist
Includes overlap size of Gencode elements

In [60]:
%%bash
head gencode_mapped.bed

chr1	16140	16200|chr1	15947	16436	intron|60
chr1	51868	52040|chr1	36081	52473	intergenic|172
chr1	57280	57354|chr1	53473	57598	intergenic|74
chr1	66370	66482|chr1	65419	66419	promoter;chr1	66419	69037	intron|49;63
chr1	79100	79231|chr1	71585	89295	intergenic|131
chr1	79430	79497|chr1	71585	89295	intergenic|67
chr1	79580	79760|chr1	71585	89295	intergenic|180
chr1	87220	87295|chr1	71585	89295	intergenic|75
chr1	88220	88360|chr1	71585	89295	intergenic|140
chr1	88700	88814|chr1	71585	89295	intergenic|114


## Choose the Best Annotation
Annotated DHS as promoter, exon, intron, or intergenic depending on which element had the largest overlap. If there was a tie, the winning annotation is based on the rank

*Rank: Promoter > Exon > Intron > Intergenic*

In [61]:
%%bash

biggest=0
col=0

sed 's/intergenic/1/g' gencode_mapped.bed \
| sed 's/intron/2/g' \
| sed 's/exon/3/g' \
| sed 's/promoter/4/g' \
> choose_best_annotation.bed

awk -F'|' -v b=$biggest -v c=$col '{
        line=$3
        split(line,a,";")

        mapped=$2
        split(mapped,m,";")
        
    if (length(a) == 1) {
        print $1"\t"$2
    }
    else {
        for(i=1;i<=NF;i++) {
            if (a[i] > b) {
                b=a[i];
                c=i;
            }
            else if (a[i] == b) {
                old=m[c];
                split(old,o,"\t");
                new=m[i];
                split(new,n,"\t");
                if (o[4] < n[4]) {
                    b=a[i];
                    c=i;
                }
            }
        }
    print $1"\t"m[c];
    b=0;
    }

}'  choose_best_annotation.bed > best_annotation.bed



awk '{print $1"\t"$2"\t"$3"\t"$7}' best_annotation.bed \
| sort-bed - \
| awk '{if($4 == 1) print $1"\t"$2"\t"$3"\t""intergenic"; else if($4 == 2) print $1"\t"$2"\t"$3"\t""intron"; else if($4 == 3) print $1"\t"$2"\t"$3"\t""exon"; else if($4 == 4) print $1"\t"$2"\t"$3"\t""promoter"}' - \
> dhs_annotated_gencode28.bed


### Annotated DHS Masterlist as Promoter, Exon, Intron, or Intergenic

In [62]:
%%bash
head dhs_annotated_gencode28.bed
cut -f4 dhs_annotated_gencode28.bed | sort - | uniq -c
total=`wc -l dhs_annotated_gencode28.bed | cut -d' ' -f1`
echo "Total: $total"

chr1	16140	16200	intron
chr1	51868	52040	intergenic
chr1	57280	57354	intergenic
chr1	66370	66482	intron
chr1	79100	79231	intergenic
chr1	79430	79497	intergenic
chr1	79580	79760	intergenic
chr1	87220	87295	intergenic
chr1	88220	88360	intergenic
chr1	88700	88814	intergenic
 158527 exon
1376951 intergenic
1891595 intron
 164825 promoter
Total: 3591898


### Parse Gencode File again but this time seperate into protein-coding and non-protein-coding regions

In [63]:
%%bash
#Filter Initial Gencode file based on protein coding and non-protein coding regions
grep protein_coding gencode.v28.basic.annotation.gtf \
| awk '{print $1"\t"$4"\t"$5"\t""PC"}' - \
| awk -F'\t' '{if($2 != $3) print}' - \
| sort-bed - \
> PC.bed

grep -v protein_coding gencode.v28.basic.annotation.gtf \
| tail -n +6 \
| awk '{print $1"\t"$4"\t"$5"\t""NPC"}' \
| awk -F'\t' '{if($2 != $3) print}' - \
| sort-bed - \
> NPC.bed

bedops -u PC.bed NPC.bed > PC-NPC-gencode.bed

### Protein coding and non-protein coding bed file

In [64]:
%%bash
head PC-NPC-gencode.bed

chr1	11869	12227	NPC
chr1	11869	14409	NPC
chr1	11869	14409	NPC
chr1	12010	12057	NPC
chr1	12010	13670	NPC
chr1	12179	12227	NPC
chr1	12613	12697	NPC
chr1	12613	12721	NPC
chr1	12975	13052	NPC
chr1	13221	13374	NPC


### Split Exon Annotated DHSs to CDS, UTR, and non-coding

1. Extract Exonic regions from DHS Annotation
2. Extract UTR and CDS labeled regions from Gencode
3. Map UTR/CDS regions to Exonic DHS Annotations
4. Pick Element with the largest overlap. If there is a tie, pick the element with 
the largest fraction of overlap
5. If Exon Annotation is not UTR or CDS, then the DHS is non-protein-coding (NPC)


In [65]:
%%bash
#Map Exon regions to DHS Annotations
awk '{if($4 == "exon") print}' dhs_annotated_gencode28.bed > dhs-exon.bed
bedops -u utr.bed cds.bed > utr-cds-gencode.bed
bedmap --echo --echo-map --echo-overlap-size --echo-map-size --ec dhs-exon.bed  utr-cds-gencode.bed \
> exon_mapped.bed

#Choose the element with the largest overlap or the largest fraction of overlap
biggest=0
col=0
fraction=0

awk -F'|' -v f=$fraction -v b=$biggest -v c=$col '{
        line=$3
        split(line,a,";")
        mapped=$2
        split(mapped,m,";")
        size=$4
        split(size,s,";")
        
       
       if (length(a) == 1) {
                c=1;
        }
        else if(length(a) > 1) {
                for(i=1;i<=NF;i++) {
                        if (a[i] > b) {
                                b=a[i];
                                c=i;
                                f=a[i]/s[i];
                        }
                        else if (a[i] == b) {
                                if(a[i]/s[i] > f) {
                                        b=a[i];
                                        c=i;
                                }
                        } 
                }
        }
        else {
            c=1;
        }
        print $1"\t"m[c];
        b=0;      
}' exon_mapped.bed > best_exon_mapped.bed


awk '{print $1"\t"$2"\t"$3"\t"$4"\t"$8}' best_exon_mapped.bed \
| sort-bed - \
| awk -F'\t' '{if($5 == "") print $1"\t"$2"\t"$3"\t"$4"\t""NPC"; else print}' \
> dhs_annotated_exon.bed

### Annotated Exonic Regions

In [66]:
%%bash
head dhs_annotated_exon.bed
cut -f5 dhs_annotated_exon.bed | sort - | uniq -c
total=`wc -l dhs_annotated_exon.bed | cut -d' ' -f1`
echo "Total: $total"

chr1	89780	89959	exon	NPC
chr1	630500	630707	exon	NPC
chr1	632295	632424	exon	NPC
chr1	632360	632511	exon	NPC
chr1	634520	634740	exon	NPC
chr1	727161	727287	exon	NPC
chr1	729346	729440	exon	NPC
chr1	729379	729468	exon	NPC
chr1	818686	818860	exon	NPC
chr1	818880	818966	exon	NPC
  68648 CDS
  28971 NPC
  60908 UTR
Total: 158527


### Split Promoter annotated DHSs to coding-gene and non-coding

1. Extract Promoter regions from DHS Annotation
2. Map protein-coding/non-coding gencode regions to Promoter DHS Annotations
3. Pick element with the largest overlap. If there is a tie, pick the element with the largest fraction of overlap

Notes:
NPC = non-coding, PC = protein-coding

In [67]:
%%bash
#Map
awk '{if($4 == "promoter") print}' dhs_annotated_gencode28.bed > dhs-promoter.bed

bedmap --echo --echo-map --echo-overlap-size --echo-map-size --skip-unmapped --ec dhs-promoter.bed PC-NPC-gencode.bed \
> promoter_mapped.bed

#Pick the element with the largest overlap or the largest fraction of overlap

biggest=0
col=0
fraction=0

awk -F'|' -v f=$fraction -v b=$biggest -v c=$col '{
        line=$3
        split(line,a,";")
        mapped=$2
        split(mapped,m,";")
        size=$4
        split(size,s,";")
        
        if (length(a) == 1) {
                c=1;
        }
        else {
                for(i=1;i<=NF;i++) {
                        if (a[i] > b) {
                                b=a[i];
                                c=i;
                                f=a[i]/s[i];
                        }
                        else if (a[i] == b) {
                                if(a[i]/s[i] > f) {
                                        b=a[i];
                                        c=i;
                                }
                        } 
                }      
        }
        print $1"\t"m[c];
        b=0;      
}' promoter_mapped.bed > best_promoter_mapped.bed

awk '{print $1"\t"$2"\t"$3"\t"$4"\t"$8}' best_promoter_mapped.bed \
| sort-bed - \
> dhs_annotated_promoter.bed

In [68]:
%%bash
head dhs_annotated_promoter.bed
cut -f5 dhs_annotated_promoter.bed | sort - | uniq -c
total=`wc -l dhs_annotated_promoter.bed | cut -d' ' -f1`
echo "Total: $total"

chr1	90140	90209	promoter	NPC
chr1	135100	135144	promoter	NPC
chr1	182681	182819	promoter	NPC
chr1	186960	187129	promoter	NPC
chr1	629100	629280	promoter	NPC
chr1	629160	629310	promoter	NPC
chr1	629512	629580	promoter	NPC
chr1	629520	629596	promoter	NPC
chr1	629870	630020	promoter	NPC
chr1	630075	630240	promoter	NPC
  47219 NPC
 112242 PC
Total: 159461


### Split Intronic annotated DHSs to coding-gene and non-coding

1. Extract Intronic regions from DHS Annotation
2. Map protein-coding/non-coding gencode regions to Intronic DHS Annotations
3. Pick element with the largest overlap. If there is a tie, pick the element with the largest fraction of overlap

Notes:
NPC = non-coding, PC = protein-coding

In [69]:
%%bash

#Map PC/NPC regions to Intronic DHSs
awk -F'\t' '{if($4 == "intron") print}' dhs_annotated_gencode28.bed > dhs-intron.bed

bedmap --echo --echo-map --echo-overlap-size --echo-map-size --skip-unmapped --ec dhs-intron.bed PC-NPC-gencode.bed \
> intron_mapped.bed

biggest=0
col=0
fraction=0

#Choose Best Annotation
awk -F'|' -v f=$fraction -v b=$biggest -v c=$col '{
        line=$3
        split(line,a,";")
        mapped=$2
        split(mapped,m,";")
        size=$4
        split(size,s,";")
        
        if (length(a) == 1) {
                c=1;
        }
        else {
                for(i=1;i<=NF;i++) {
                        if (a[i] > b) {
                                b=a[i];
                                c=i;
                                f=a[i]/s[i];
                        }
                        else if (a[i] == b) {
                                if(a[i]/s[i] > f) {
                                        b=a[i];
                                        c=i;
                                }
                        } 
                }      
        }
        print $1"\t"m[c];
        b=0;      
}' intron_mapped.bed > best_intron_mapped.bed

awk '{print $1"\t"$2"\t"$3"\t"$4"\t"$8}' best_intron_mapped.bed \
| sort-bed - \
> dhs_annotated_intron.bed

In [70]:
%%bash
head dhs_annotated_intron.bed
cut -f5 dhs_annotated_intron.bed | sort - | uniq -c
total=`wc -l dhs_annotated_intron.bed | cut -d' ' -f1`
echo "Total: $total"

chr1	16140	16200	intron	NPC
chr1	66370	66482	intron	PC
chr1	99630	99717	intron	NPC
chr1	113860	113950	intron	NPC
chr1	128619	128757	intron	NPC
chr1	186727	186834	intron	NPC
chr1	186817	186996	intron	NPC
chr1	190865	190920	intron	NPC
chr1	190920	191071	intron	NPC
chr1	191260	191340	intron	NPC
 383039 NPC
1508556 PC
Total: 1891595


### Clean-up Directories

In [73]:
%%bash

#Move Everything into extra_files initially
mkdir -p extra_files
mv *bed extra_files
mv *gtf extra_files
mv *txt extra_files

#Raw Files
mkdir -p raw_files
mv extra_files/gencode.v28.basic.annotation.gtf raw_files
mv extra_files/chromInfo.hg38.bed raw_files
mv extra_files/DHS_Index_and_Vocabulary_hg38_WM20190703.txt raw_files

#Results
mkdir -p results
mv extra_files/dhs_annotated_*.bed results

